# PyTorch基本操作实验

## 一、Pytorch基本操作考察

### 1、使用 𝐓𝐞𝐧𝐬𝐨𝐫 初始化一个 𝟏×𝟑 的矩阵 𝑴 和一个 𝟐×𝟏 的矩阵 𝑵，对两矩阵进行减法操作（要求实现三种不同的形式），给出结果并分析三种方式的不同（如果出现报错，分析报错的原因），同时需要指出在计算过程中发生了什么

In [33]:
import torch
M = torch.tensor([1,2,3])
N = torch.tensor([[1],[1]])

In [34]:
# 方式一：直接相减
res1 = M - N
print("方式一结果:",'\n',res1)
# PyTorch启用广播机制，它自动调整N的形状以匹配M的形状。在这里，N被复制并扩展成2x3的矩阵
# M被扩展成2x3矩阵然后进行相减

方式一结果: 
 tensor([[0, 1, 2],
        [0, 1, 2]])


In [35]:
# 方式二：torch.sub进行
res2 = M.sub(N)
print("方式二结果:",'\n',res2)
# 方式二同样也利用了广播机制，与直接相减方式相同

方式二结果: 
 tensor([[0, 1, 2],
        [0, 1, 2]])


In [36]:
# 方式三：inplace函数
import torch
M = torch.tensor([1,2,3])
N = torch.tensor([[1],[1]])
res3 = M.sub_(N)
print("方式三结果:",'\n',res3)
# 由于广播机制会扩大张量M的在一维和二维上的长度，与inplace原地操作不兼容，所以会报错

RuntimeError: output with shape [3] doesn't match the broadcast shape [2, 3]

### 2、① 利用 𝐓𝐞𝐧𝐬𝐨𝐫 创建两个大小分别 𝟑×𝟐 和 𝟒×𝟐 的随机数矩阵 𝑷 和 𝑸 ，要求服从均值为0，标准差0.01为的正态分布；② 对第二步得到的矩阵 𝑸 进行形状变换得到 𝑸 的转置 𝑸^𝑻 ；③ 对上述得到的矩阵 𝑷 和矩阵 𝑸^𝑻 求矩阵相乘

In [38]:
# 问题1解答：
import torch
P = torch.normal(0,0.01,(3,2))
Q = torch.normal(0,0.01,(4,2))
print(P,'\n',Q)

tensor([[ 0.0074, -0.0048],
        [-0.0020, -0.0115],
        [ 0.0093, -0.0224]]) 
 tensor([[ 7.8963e-03, -2.2350e-03],
        [ 2.6952e-03,  2.3349e-02],
        [-9.6502e-05,  1.4392e-02],
        [-6.8178e-04, -7.6675e-04]])


In [41]:
# 问题二解答：获得Q的转置
Q = Q.T
print(Q)

tensor([[ 7.8963e-03,  2.6952e-03, -9.6502e-05, -6.8178e-04],
        [-2.2350e-03,  2.3349e-02,  1.4392e-02, -7.6675e-04]])


In [50]:
# 问题三解答：矩阵相乘
torch.mm(P,Q)

tensor([[ 6.9028e-05, -9.2042e-05, -6.9715e-05, -1.3586e-06],
        [ 9.6982e-06, -2.7486e-04, -1.6584e-04,  1.0235e-05],
        [ 1.2397e-04, -4.9866e-04, -3.2380e-04,  1.0828e-05]])

### 3、给定公式 𝑦_3=𝑦_1+𝑦_2=𝑥^2+𝑥^3，且 𝑥=1。利用学习所得到的Tensor的相关知识，求𝑦_3对𝑥的梯度，即(𝑑𝑦_3)/𝑑𝑥。要求在计算过程中，在计算〖 𝑥〗^3 时中断梯度的追踪，观察结果并进行原因分析 提示, 可使用 with torch.no_grad()， 举例: with torch.no_grad(): y = x * 5

In [59]:
x = torch.tensor(1,dtype=torch.float64,requires_grad=True)

In [61]:
y1=x*x
y2=x*x*x
y3=y1+y2
y3.backward()
print(x.grad)

tensor(5., dtype=torch.float64)


## 二、动手实现 logistic 回归

### 1、要求动手从0实现 logistic 回归（只借助Tensor和Numpy相关的库）在人工构造的数据集上进行训练和测试，并从loss以及训练集上的准确率等多个角度对结果进行分析（可借助nn.BCELoss或nn.BCEWithLogitsLoss作为损失函数，从零实现二元交叉熵为选作）

In [62]:
import torch
from torch.utils import data
import random

In [78]:
# 生成一些数据集
def generate_data(w,b,num_examples):
    X = torch.normal(0,1,(num_examples,len(w)))
    z = torch.matmul(X,w) + b
    z += torch.normal(0,0.01,z.shape)
    y = 1.0/(1.0+(-z).exp())
    y = (y > 0.5).int()
    return X,y.reshape((-1,1))
true_w = torch.tensor([[2],[-3.4]])
true_b = 4.2
features,labels = generate_data(true_w,true_b,1000)

In [79]:
def data_iter(batch_size,features,labels):
    num_examples = len(features)
    indices = list(range(num_examples))
    random.shuffle(indices)
    for i in range(0,num_examples,batch_size):
        batch_indices = torch.tensor(indices[i:min(i+batch_size,num_examples)])
        yield features[batch_indices],labels[batch_indices]
        
batch_size = 10


In [80]:
# 定义模型
def logistic(X,w,b):
    return 1.0/(1.0+(torch.matmul(X,w) + b).exp())

In [81]:
# 定义损失函数
from torch import nn
loss = nn.BCELoss()

In [82]:
# 定义优化模型
def sgd(params,lr,batch_size):
    with torch.no_grad():
        for param in params:
            param -= lr*param.grad/batch_size
            param.grad.zero_()

In [83]:
w = torch.normal(0,0.01,size = (2,1),requires_grad=True)
b = torch.zeros(1,requires_grad=True)
lr = 0.003
num_epochs=3
net = logistic
for epoch in range(num_epochs):
    for X,y in data_iter(batch_size,features,labels):
        l = loss(net(X,w,b),y.float())
        l.sum().backward()
        sgd([w,b],lr,batch_size)
    with torch.no_grad():
        train_l = loss(net(features, w, b), labels.float())
        print(f'epoch {epoch + 1}, loss {float(train_l.mean()):f}')

epoch 1, loss 0.684979
epoch 2, loss 0.679481
epoch 3, loss 0.674068


In [84]:
print(f'w的估计误差: {true_w - w.reshape(true_w.shape)}')
print(f'b的估计误差: {true_b - b}')

w的估计误差: tensor([[ 2.0258],
        [-3.4211]], grad_fn=<SubBackward0>)
b的估计误差: tensor([4.2326], grad_fn=<RsubBackward1>)


## 2、利用 torch.nn 实现 logistic 回归在人工构造的数据集上进行训练和测试，并对结果进行分析，并从loss以及训练集上的准确率等多个角度对结果进行分析

In [85]:
import numpy as np
import torch
from torch.utils import data

In [86]:
# 生成一些数据集
def generate_data(w,b,num_examples):
    X = torch.normal(0,1,(num_examples,len(w)))
    z = torch.matmul(X,w) + b
    z += torch.normal(0,0.01,z.shape)
    y = 1.0/(1.0+(-z).exp())
    y = (y > 0.5).int()
    return X,y.reshape((-1,1))
true_w = torch.tensor([[2],[-3.4]])
true_b = 4.2
features,labels = generate_data(true_w,true_b,1000)

In [ ]:
def load_array(data_array,batch_size,is_train=True):
    dataset = TensorDataset(*data_arrays)
    return data.DataLoader(dataset,batch_size,shuffle=is_train)
batch_size = 10
data_iter = load_array((features,labels),batch_size)